In [1]:
from mnist_data_loader import *
from performance_metrics import *
dataset_base_dir = 'Data'
mnist_dataset_fname = 'mnist.pkl.gz'
dataset_1_fname = 'project3_dataset1.txt'
dataset_2_fname = 'project3_dataset2.txt'

mnist_data = load_data_wrapper(dataset_base_dir+'/'+mnist_dataset_fname)

In [2]:
(train_X,train_Y), (valid_X,valid_Y), (test_X,test_Y) = mnist_data

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers

tf.random.set_seed(88)
tf.keras.utils.set_random_seed(88)

weight_initializer = tf.keras.initializers.GlorotUniform(seed=88)
input_shape = (784,1)
batch_size = 80
output_size = 10

learning_rate = 1e-3

epoch = 100

hidden_size_1, hidden_size_2 = 256, 64


In [4]:


input_layer = layers.InputLayer(
    input_shape=input_shape,
    batch_size=batch_size,
    name='MNIST_Input_Layer'
)

flatten_layer = layers.Flatten(name='Flatten_Layer')

hidden_layer_1 = layers.Dense(
    units=hidden_size_1,
    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.L2(1e-4),
    activity_regularizer=regularizers.L2(1e-5),
    activation='sigmoid',
    use_bias=True,
    kernel_initializer=weight_initializer,
    bias_initializer='zeros',
    kernel_constraint=None,
    bias_constraint=None,
    name='Hidden_Layer_1'
)
hidden_layer_2 = layers.Dense(
    units=hidden_size_2,
    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.L2(1e-4),
    activity_regularizer=regularizers.L2(1e-5),
    activation='sigmoid',
    use_bias=True,
    kernel_initializer=weight_initializer,
    bias_initializer='zeros',
    kernel_constraint=None,
    bias_constraint=None,
    name='Hidden_Layer_2'
)

output_layer = layers.Dense(
    units=output_size,
#     kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
#     bias_regularizer=regularizers.L2(1e-4),
#     activity_regularizer=regularizers.L2(1e-5),
    activation='softmax',
    use_bias=True,
    kernel_initializer=weight_initializer,
    bias_initializer='zeros',
    name = 'Output_Layer'
)

# reshape_layer = layers.Reshape((-1,1), name='Reshape_Layer')


model = tf.keras.models.Sequential([
    input_layer,
    flatten_layer,
    hidden_layer_1,
    hidden_layer_2,
    output_layer,
#     reshape_layer
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Flatten_Layer (Flatten)     (80, 784)                 0         
                                                                 
 Hidden_Layer_1 (Dense)      (80, 256)                 200960    
                                                                 
 Hidden_Layer_2 (Dense)      (80, 64)                  16448     
                                                                 
 Output_Layer (Dense)        (80, 10)                  650       
                                                                 
Total params: 218,058
Trainable params: 218,058
Non-trainable params: 0
_________________________________________________________________


In [5]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=1000,
    decay_rate=0.9)
optimizers = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam_Optimizer")

loss_fnc = tf.keras.losses.SparseCategoricalCrossentropy()


model.compile(loss=loss_fnc, optimizer=optimizers, metrics=['accuracy'])



early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=1e-7,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

my_callbacks = [
    early_stop,
    tf.keras.callbacks.ModelCheckpoint(filepath='model/model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='log'),
]


In [6]:

model.fit(
    train_X,train_Y,
    batch_size=batch_size,
    epochs=epoch,
    verbose='auto',
    callbacks=my_callbacks,
    validation_data=(valid_X,valid_Y),
    shuffle=True
)

Epoch 1/100
625/625 [==============================] - 6s 6ms/step - loss: 0.7960 - accuracy: 0.8433 - val_loss: 0.4430 - val_accuracy: 0.9200
Epoch 2/100
625/625 [==============================] - 3s 5ms/step - loss: 0.4242 - accuracy: 0.9218 - val_loss: 0.3620 - val_accuracy: 0.9374
Epoch 3/100
625/625 [==============================] - 3s 5ms/step - loss: 0.3670 - accuracy: 0.9343 - val_loss: 0.3337 - val_accuracy: 0.9455
Epoch 4/100
625/625 [==============================] - 4s 7ms/step - loss: 0.3305 - accuracy: 0.9437 - val_loss: 0.3045 - val_accuracy: 0.9555
Epoch 5/100
625/625 [==============================] - 3s 5ms/step - loss: 0.3050 - accuracy: 0.9512 - val_loss: 0.2853 - val_accuracy: 0.9570
Epoch 6/100
625/625 [==============================] - 4s 6ms/step - loss: 0.2864 - accuracy: 0.9566 - val_loss: 0.2687 - val_accuracy: 0.9624
Epoch 7/100
625/625 [==============================] - 3s 5ms/step - loss: 0.2711 - accuracy: 0.9602 - val_loss: 0.2643 - val_accuracy: 0.9635

Epoch 58/100
625/625 [==============================] - 3s 4ms/step - loss: 0.1630 - accuracy: 0.9854 - val_loss: 0.1808 - val_accuracy: 0.9785
Epoch 59/100
625/625 [==============================] - 3s 5ms/step - loss: 0.1629 - accuracy: 0.9852 - val_loss: 0.1810 - val_accuracy: 0.9786
Epoch 60/100
625/625 [==============================] - 3s 5ms/step - loss: 0.1627 - accuracy: 0.9852 - val_loss: 0.1806 - val_accuracy: 0.9792
Epoch 61/100
625/625 [==============================] - 3s 4ms/step - loss: 0.1626 - accuracy: 0.9856 - val_loss: 0.1807 - val_accuracy: 0.9783
Epoch 62/100
625/625 [==============================] - 3s 5ms/step - loss: 0.1625 - accuracy: 0.9854 - val_loss: 0.1805 - val_accuracy: 0.9787
Epoch 63/100
625/625 [==============================] - 3s 5ms/step - loss: 0.1624 - accuracy: 0.9856 - val_loss: 0.1805 - val_accuracy: 0.9791
Epoch 64/100
625/625 [==============================] - 3s 5ms/step - loss: 0.1623 - accuracy: 0.9853 - val_loss: 0.1805 - val_accuracy:

In [7]:
pred_Y = model.predict(test_X)


313/313 [==============================] - 1s 2ms/step


In [8]:
np.sum(pred_Y==test_Y)/10000

C:\Users\nibir\AppData\Local\Temp\ipykernel_24204\3053982549.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.sum(pred_Y==test_Y)/10000


0.0

In [13]:
calculate_performance(pred_Y, test_Y)

{'accuracy': 0.9775,
 'precision_macro': 0.97745007981557,
 'precision_micro': 0.9775,
 'precision_weighted': 0.9775033097726747,
 'recall_macro': 0.9772401677387043,
 'recall_micro': 0.9775,
 'recall_weighted': 0.9775,
 'f1_macro': 0.9773267733758197,
 'f1_micro': 0.9775,
 'f1_weighted': 0.9774836477555034,
 'roc_auc': 0.9996574852932418}